# Parsing and Cleaning PHEME RNR Dataset Events

This notebook cleans tweet level data generated from `lib/pheme_parsing.py` and aggregates this tabular, individual data to tabular thread-level data. It also provides a useful sanity check after making modifications to `lib/pheme_parsing.py`. 

## Instructions
1. Update the variable `event` in the cell below with one of the following events:
    1. germanwings-crash
1. Run all the cells in this notebook to generate thread-level CSV files in the `data/threads` directory.

In [1]:
# Load dependencies for this Jupyter Notebook
import pandas as pd
import numpy as np
import networkx as nx
from functools import reduce
from lib.util import fetch_tweets

event = "germanwings-crash"  # Change this value to clear different PHEME datasets

## Parsing and Cleaning Data
This step takes the raw PHEME rumor dataset and saves it tabular format as CSV file. The original PHEME dataset consists of JSON files organized into directories by event and category (rumor or non-rumor). These three functions below parse the data, save it as a CSV file (if necessary), and load it into this notebook as a Pandas DataFrame from the "cached" CSV file.

In [2]:
gw = fetch_tweets(event)

##  Tweet Level Features

| Name/Column       | Description                   | Type   | Notes  |
|-------------------|-------------------------------|--------| ------ |
| is_rumor          | Was this classified as rumor  | "bool" (`int`) | *Classification done by journalists* |
| thread            | Source tweet id               | `str`  |                                                   |
| in_reply_tweet    | Tweet ID in reply to          | `str`  |                                                   |
| event             | Name of the PHEME event       | `str`  | Corresponds to event in the PHEME dataset         |
| tweet_id          | Unique ID for tweet           | `str`  | This field is the ID referenced in `in_reply_tweet`     |
| is_source_tweet   | Was this classified as rumor  | "bool" (`int`) |                                                   |
| in_reply_user     | User ID in reply to           | `str`  |                                                   |
| user_id           | Twitter User's ID             | `str`  | This field is the ID referenced in `in_reply_user` |
| tweet_length      | Number of characters in tweet | `int`  |                                                   |
| urls_count        | Number of URLS in tweet       | `int`  |                                                   |
| hashtags_count    | Number of hashtags in tweet   | `int`  |                                                   |
| retweet_count     | Times the tweet was retweeted | `int`  |                                                   |
| favorite_count    | Number of times favorited     | `int`  |                                                   |
| mentions_count    | Number of users mentioned     | `int`  |                                                   |
| is_truncated      | Is this tweet truncated       | "bool" (`int`) | Did User type > 140 characters. [See Tweet updates](https://developer.twitter.com/en/docs/tweets/tweet-updates) |
| created              | Datetime Tweet was created    | `datetime` | |
| has_smile_emoji      | Does Tweet contain ""?        | "bool" (`int`) | 😊 is the smile emoji |
| user.tweets_count    | User's tweet total, currently | `int`  | |
| user.verified        | Is Twitter user verified?     | "bool" (`int`) |                                                   |
| user.followers_count | Total number of followers  | `int` | |
| user.listed_count    | ?? | `int` | | 
| user.friends_count   | ?? | `int` | |
| user.time_zone       | Timezone of the user's Twitter account | `str` | |
| user.desc_length     | Length of the user's biographic description | `int` |
| user.has_bg_img      | Does user have a profile background image?  | "bool" (`int`) |
| user.default_pric    | Does the user have the default profile picture | "bool" (`int`) |
| user.created_at      | Date and time Twitter account was activated | `datetime` | |
| user.utc_dist        | TK | `int` | See [this blog post time and the Twitter API](https://zacharyst.com/2017/04/05/assigning-the-correct-time-to-a-twee) |

## Germanwings Crash

In [3]:
gw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4489 entries, 0 to 999
Data columns (total 46 columns):
is_rumor                4489 non-null int64
thread                  4489 non-null int64
in_reply_tweet          4019 non-null float64
event                   4489 non-null object
tweet_id                4489 non-null int64
is_source_tweet         4489 non-null int64
in_reply_user           4020 non-null float64
user_id                 4489 non-null int64
tweet_length            4489 non-null int64
symbol_count            4489 non-null int64
user_mentions           4489 non-null int64
urls_count              4489 non-null int64
media_count             4489 non-null int64
hashtags_count          4489 non-null int64
retweet_count           4489 non-null int64
favorite_count          4489 non-null int64
mentions_count          4489 non-null int64
is_truncated            4489 non-null int64
created                 4489 non-null int64
has_smile_emoji         4489 non-null int64
sensitive

The `.head` method prints out the 5 first rows in the dataframe

In [4]:
gw.head()

,is_rumor,thread,in_reply_tweet,event,tweet_id,is_source_tweet,in_reply_user,user_id,tweet_length,symbol_count,...,user.notifications,user.friends_count,user.time_zone,user.has_bg_img,user.default_pic,user.created_at,user.location,user.profile_sbcolor,user.profile_bgcolor,user.utc_dist
0,0,580319983676313600,NaN,germanwings-crash,580319983676313601,1,NaN,8330472,98,0,...,0,74,Madrid,1,0,2007-08-21 14:03:19,1,16777215,11453380,0.0
1,0,580319983676313600,5.803200e+17,germanwings-crash,580322851850461184,0,8.330472e+06,2307392966,109,0,...,0,1782,None,1,1,2014-01-23 23:26:57,1,12639981,12639981,NaN
10,0,580322102307373056,5.803221e+17,germanwings-crash,580349597937213440,0,1.621157e+07,40643525,143,0,...,0,63,Athens,1,1,2009-05-17 11:10:29,1,12639981,12639981,2.0
100,0,580324804403818496,5.803700e+17,germanwings-crash,580377215667310592,0,6.143144e+07,3028982049,132,0,...,0,218,None,1,1,2015-02-10 23:27:50,1,12639981,12639981,NaN
1000,0,580341214098137088,5.803412e+17,germanwings-crash,580353017402245121,0,1.453758e+09,2843470199,97,0,...,0,11,None,1,1,2014-10-25 10:29:55,0,12639981,12639981,NaN


## Boolean Columns

The `describe` method will give summary information about each column in the dataframe. Each of these columns, except `is_truncated` should have two unique values.

Just for a sanity check. The cell below converts these boolean columns into value of type `bool` and describes them.

In [5]:
bool_columns = ["is_rumor", "is_source_tweet", "is_truncated", 
                "has_smile_emoji", "user.verified", "user.has_bg_img", 
                "user.default_pic", "sensitive", "has_place", "has_coords", "user.notifications"]

gw[bool_columns].astype(bool).describe(include="bool")

,is_rumor,is_source_tweet,is_truncated,has_smile_emoji,user.verified,user.has_bg_img,user.default_pic,sensitive,has_place,has_coords,user.notifications
count,4489,4489,4489,4489,4489,4489,4489,4489,4489,4489,4489
unique,2,2,1,2,2,2,2,2,2,2,1
top,True,False,False,False,False,True,False,False,False,False,False
freq,2494,4020,4489,4487,4109,3992,2827,4455,4228,4363,4489


Some columns in some PHEME events have only one unique value for all tweets. So we'll drop any where they exist.

In [6]:
# for col in gw.columns:
#     if len(gw[col].unique()) == 1:
#         print("Dropping %s\n" % col)
#         gw.drop(col, inplace=True, axis = 1)

## Thread Level Features

* **Bold features** represent high performing features identified in C. Buntain and J. Golbeck, ["Automatically Identifying Fake News in Popular Twitter Threads"](http://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8118443&isnumber=8118402)
* Features that are normalized are normalized by thread length


| Name                | Description                               | Type    | Notes |
| ---                 | ---                                       | ---     | ----- |
| thread              | Tweet ID of the source tweet              | `str`   | |
| favorite_count      | Normalized favorite total                 | `float` | |
| retweet_count       | Normlaized retweet total                  | `float` | |
| **hashtags_count**  | Normlaized hashtag total                  | `float` | |
| urls_count          | URL total normalized by thread length     | `float`  | |
| user.tweets_count   | Total tweets by thread users              | `float` | |
| event               | Name of PHEME event                       | `str`  | |
| is_rumor            | Either rumor or nonrumor                  | `bool` | |
| thread_length       | Number of tweets in the thread            | `int`  | |
| user.has_bg_img     | Ratio of users who have bg image          | `float`| |
| user.default_pic    | Ratio of users with default profile pic   | `float`| |
| **has_smile_emoji** | Number of smile emojis in the thread      | `int`  | 😊 is the smile emoji |
| user.verified       | Count of verified users in the thread normalized by thread length     | `float`  | |
| **src.followers_count** | The number of followers of the original poster of the thread. | `int` | |
| src.listed_count    | TODO | `int` | |
| src.user_verified   | TODO | `int` | |
| src.tweets_total    | TODO | `int` | |
| reply_var           | The variance in the timestamps of responses to the source tweet | `float` |
| src_age             | Difference in src user's creation and tweet creation            | `int`   | Measured in seconds |
| time_to_first_resp  | The difference between tweet creation datetime and 1st reply    | `int`   | Measured in seconds |
| time_to_last_resp   | The difference between tweet creation datetime and last reply   | `int`   | Measured in seconds |

In [7]:

def agg_tweets_by_thread(df):
    
    shared = lambda x: 1 - len(set(x)) / len(x)
    shared.__name__ = "shared"

    # Returns the proportion of True/1 values in col
    funcs = [np.mean, sum, np.var]
    agg_props = {
        "favorite_count": funcs,
        "user_mentions": funcs,
        "media_count": funcs,
        "sensitive": funcs,
        "has_place": funcs,
        "has_coords": funcs,
        "retweet_count": funcs,
        "hashtags_count": funcs + [shared],
        "urls_count": funcs,
        "user.tweets_count": funcs,
        "is_rumor": max,
        "tweet_id": len,
        "user.has_bg_img": funcs,
        "has_quest": funcs,
        "has_exclaim": funcs,
        "has_quest_or_exclaim": funcs,
        "user.default_pic": funcs,
        "has_smile_emoji": funcs,
        "user.verified": funcs,
        "user.name_length": funcs,
        "user.handle_length": funcs,
        "user.profile_sbcolor": funcs,
        "user.profile_bgcolor": funcs,
    }
    rename = {
        "tweet_id": "thread_length"
    }

    def g(x):
        # Add size of largest user-to-user conversation component in each thread        
        d = []
        thread_tweets = list(x["tweet_id"])
        G = nx.from_pandas_edgelist(df[df.tweet_id.isin(thread_tweets)], "user_id", "in_reply_user")
        Gc = max(nx.connected_component_subgraphs(G), key=len)
        d.append(nx.number_connected_components(G))
        d.append(nx.diameter(Gc))
        return pd.Series(d, index=["component_count", "largest_cc_diameter"])
    
    # Step 0: Build graph-based features
    graph = df.groupby("thread").apply(g)
    
    # Step 1: Build simple aggregate features
    agg = df.groupby("thread")\
        .agg(agg_props)\
        .rename(columns=rename)
    
    agg.columns = [ "_".join(x) for x in agg.columns.ravel() ]
    agg = agg.rename(columns={"is_rumor_max": "is_rumor", "thread_length_len": "thread_length"})
    
    # Step 2: Builds some features off the source tweet, which has tweet_id == thread            
    src = df[df["is_source_tweet"] == 1][["thread",
                                          "user.followers_count", 
                                          "user.listed_count",
                                          "user.verified",
                                          "created",
                                          "user.created_at",
                                          "user.tweets_count"]] \
                         .rename(columns={"user.followers_count": "src.followers_count",
                                          "user.listed_count": "src.listed_count",
                                          "user.verified": "src.user_verified",
                                          "user.created_at": "src.created_at",
                                          "user.tweets_count": "src.tweets_total"})
    
    # Step 3: Build features off of the reply tweets
    def f(x):
        d = []
        
        # Get various features from the distribution of times of reply tweet
        d.append(min(x["created"]))
        d.append(max(x["created"]))
        d.append(np.var(x["created"]))
                
        return pd.Series(d, index=["first_resp", "last_resp","resp_var"])
        
    replies = df[df["is_source_tweet"] == False] \
        .groupby("thread") \
        .apply(f)

    graph_features = df.groupby("thread").apply(g)
    
    dfs = [agg, src, replies, graph]
    thrd_data = reduce(lambda left, right: pd.merge(left,right, on="thread"), dfs)
    
    # Step 3: Add miscelaneous features
    # Remember timestamps increase as time progresses
    # src.created_at < created < first_resp < last_resp
    thrd_data["time_to_first_resp"] = thrd_data["first_resp"] - thrd_data["created"]
    thrd_data["time_to_last_resp"] = thrd_data["last_resp"] - thrd_data["created"]
    
    return thrd_data

In [8]:
gw_thrds = agg_tweets_by_thread(gw)
gw_thrds.columns

Index(['thread', 'favorite_count_mean', 'favorite_count_sum',
       'favorite_count_var', 'user_mentions_mean', 'user_mentions_sum',
       'user_mentions_var', 'media_count_mean', 'media_count_sum',
       'media_count_var', 'sensitive_mean', 'sensitive_sum', 'sensitive_var',
       'has_place_mean', 'has_place_sum', 'has_place_var', 'has_coords_mean',
       'has_coords_sum', 'has_coords_var', 'retweet_count_mean',
       'retweet_count_sum', 'retweet_count_var', 'hashtags_count_mean',
       'hashtags_count_sum', 'hashtags_count_var', 'hashtags_count_shared',
       'urls_count_mean', 'urls_count_sum', 'urls_count_var',
       'user.tweets_count_mean', 'user.tweets_count_sum',
       'user.tweets_count_var', 'is_rumor', 'thread_length',
       'user.has_bg_img_mean', 'user.has_bg_img_sum', 'user.has_bg_img_var',
       'has_quest_mean', 'has_quest_sum', 'has_quest_var', 'has_exclaim_mean',
       'has_exclaim_sum', 'has_exclaim_var', 'has_quest_or_exclaim_mean',
       'has_quest_o

In [9]:
gw_thrds.head()

,thread,favorite_count_mean,favorite_count_sum,favorite_count_var,user_mentions_mean,user_mentions_sum,user_mentions_var,media_count_mean,media_count_sum,media_count_var,...,created,src.created_at,src.tweets_total,first_resp,last_resp,resp_var,component_count,largest_cc_diameter,time_to_first_resp,time_to_last_resp
0,580317556516483072,5.000000,40,167.714286,2.000000,16,8.285714,0.125,1,0.125,...,1427193461000,2014-01-07 11:38:00,14408,1.427193e+12,1.427194e+12,2.810739e+10,2,2,36000.0,590000.0
1,580317998147325952,0.666667,2,1.333333,0.666667,2,0.333333,0.000,0,0.000,...,1427193567000,2009-08-20 15:42:13,87411,1.427194e+12,1.427194e+12,1.600000e+09,1,2,101000.0,181000.0
2,580318020192571392,0.500000,1,0.500000,0.500000,1,0.500000,0.000,0,0.000,...,1427193572000,2007-05-08 16:02:23,22201,1.427198e+12,1.427198e+12,0.000000e+00,1,2,4476000.0,4476000.0
3,580318210609696768,4.500000,9,40.500000,1.000000,2,2.000000,0.000,0,0.000,...,1427193617000,2009-03-26 12:33:47,13875,1.427194e+12,1.427194e+12,0.000000e+00,1,2,193000.0,193000.0
4,580318669483413504,1.800000,9,16.200000,0.800000,4,0.200000,0.000,0,0.000,...,1427193727000,2009-09-29 13:06:54,5426,1.427198e+12,1.427218e+12,5.194105e+13,1,2,4258000.0,24336000.0


In [10]:
gw_thrds.shape

(405, 80)

In [11]:
fn = "data/threads/%s.csv" % event
gw_thrds.to_csv(fn, index=False)
"Wrote data to %s" % fn

'Wrote data to data/threads/germanwings-crash.csv'